### LIBRARY IMPORT, CLASS 정의

In [2]:
import pandas as pd
import numpy as np
from scipy.optimize import linprog
from tqdm import tqdm
from collections import defaultdict
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning) # FutureWarning 제거
pd.set_option('mode.chained_assignment',  None) # <==== 경고를 끈다

from utils_ver33 import *

In [3]:
from sqlalchemy import create_engine, text
import os
import logging
import cx_Oracle
import time

start = time.time()
# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

In [4]:
output_dir='/home/hyungshin.lee/dbqm_data/input'

In [3]:
start_date='2017-01-01'

In [11]:
db = QuantDBconnect()
KOSPI200 = db.GetAllKOSPI200()

tradingDates = np.unique(KOSPI200['Dates'].values.flatten())
code = np.unique(KOSPI200['Symbol'].values.flatten())

price = pd.read_pickle(output_dir+'/dataFinal/AdjPrc.pkl')
transaction = pd.read_pickle(output_dir+'/dataFinal/Amt_Avg_20D.pkl')
consensus = pd.read_pickle(output_dir+'/dataFinal/Num_EPS_ESts_E3.pkl')
IFRSsector = pd.read_pickle(output_dir+'/dataFinal/FG_IndustryGroup_Idx.pkl')
# indexCapTotal = pd.read_pickle(output_dir+'/dataFinal/indexCapTotal.pkl').sort_index()
MktCap = pd.read_pickle(output_dir+'/dataFinal/MktCap.pkl')
size = pd.read_pickle(output_dir+'/dataFinal/KSECapSize.pkl')

# indexCapTotal = indexCapTotal.reset_index().drop_duplicates(['Dates', 'Symbol'])
EoM = GetEndOfMonth(pd.DataFrame(tradingDates, columns = ['Dates']))

In [12]:
# date_temp_start = pd.to_datetime('2011-12-28')
# date_temp_end = pd.to_datetime('2024-06-30')

# indexCapTotal = indexCapTotal.set_index(['Dates', 'Symbol'])
# tradingDatesInterval = tradingDates[(date_temp_start <= tradingDates) & (tradingDates <= date_temp_end)]

REGIMEDIR = output_dir+"/dataFinal/regime.xlsx"
ESGDATADIR = output_dir+"/dataFinal/ESGUniverse_upload_total.xlsx"
EntireDataDir = output_dir+'/dataFinal/'

# SelectedFactors = ['BP','SalesP','AssetP','ROE','EBITDAY','EY_12M','EPSG','OPG','DY','InstituitionalBuy','Leverage','OPM','OP_M1M','PriceMomentum','ReverseMomentum']
SelectedFactors = ['BP','SalesP','EY_12M','OPG','DY','InstituitionalBuy','OP_M1M','PriceMomentum','ReverseMomentum']

DynamicPortfolio = False

In [13]:
DataFrameDict = CallEntireData(EntireDataDir)
ratioDataFrame = GetAllratio(tradingDates, EoM)

In [14]:
df_ls=[DataFrameDict[col].set_index(['Dates', 'Symbol']).drop(columns=['Name']) for col in DataFrameDict.keys()]

In [15]:
df_final=pd.concat(df_ls, axis=1)

In [16]:
# df_final.to_parquet(output_dir+f'/DQ_factor_{}.parquet')

In [17]:
DataFrameDict['AdjPrc'].set_index(['Dates', 'Symbol']).drop(columns=['Name'])

AdjPrc
Dates      Symbol          
2010-01-26 A000010      NaN
           A000020   6130.0
           A000030      NaN
           A000040  10401.0
           A000050   8781.0
...                     ...
2025-05-02 A900140   1720.0
           A950010      NaN
           A950070      NaN
           A950100      NaN
           A950210  14130.0

[4516744 rows x 1 columns]

In [18]:
Dynamicportfolio = False # 무시
A005930Ratio = Get005930Ratio(MktCap)

In [19]:
date_temp_start = pd.to_datetime('2011-12-29')
# date_temp_end = pd.to_datetime('2024-06-28')

In [20]:
# tradingDatestemp = tradingDates[(date_temp_start <= tradingDates) & (tradingDates <= date_temp_end)]

In [21]:
# factor_ret_full = pd.read_excel('./dataFinal/full_factor_1030.xlsx')

# factor_ret_full.columns = ['Dates'] + list(factor_ret_full.columns[1:])
# factor_ret_full = factor_ret_full.set_index('Dates')
# factor_ret_full = factor_ret_full.loc[date_temp_start:,:]

In [22]:
# REGIMEDIR = "./dataFinal/regime.xlsx"
# ESGDATADIR = "./dataFinal/ESGUniverse_upload_total.xlsx"

In [23]:
# date = pd.Timestamp('2024-10-30')

### 국면 데이터 읽어오기

In [24]:
# RegimeTemp = pd.read_excel(REGIMEDIR)            
# RegimeTemp.columns = ['date', 'RegimeTemp']
# RegimeTemp = RegimeTemp.set_index('date')

In [25]:
# RegimeDF = GetRegime(tradingDates, REGIMEDIR)

### ESG 데이터 읽어오기

In [26]:
# ESGDF = LoadESG(date, ESGDATADIR)

In [27]:
EsgScoreSustin = {'AA':50,'A':40,'BB':30,'B':20,'C':10,'D':0,'E':0,np.nan:0,0:0}
EsgScoreDB = {'S':'AA','AA':'AA','BB':'BB','A':'A','B':'BB','C':'B','D':'C','E':np.nan,'-':np.nan, np.nan: np.nan, 0:0}

In [28]:
# kospi200atD = GetKOSPI200AtD(KOSPI200, date)
# A005930Weight = A005930Ratio.loc[date, 'ratio']
# ratio = ratioDataFrame.loc[date, 'Ratio']

# DataTable = GetDataTable(DataFrameDict, tradingDates, date, Dynamicportfolio, kospi200atD)
# RawTable = GetRawDataTable(DataTable, ratio)

# dd = date + np.timedelta64(-60, "D")
# dd = GetBusinessDayBefore(tradingDates, dd)
# flag2yr = 0
# if pd.Timestamp(date).year == pd.Timestamp(dd).year:
#     FactorTable = GetFactorInitialTable(RawTable, ratio, flag2yr)
# else:
#     flag2yr = 1
#     FactorTable = GetFactorInitialTable(RawTable, ratio, flag2yr)

# FactorTable = AdjustLeverage(FactorTable)
# FactorTable = StandardizeColumns(FactorTable)

In [29]:
# factor_ret_final = factor_ret_full.loc['2012-01-02':,:]

### 16년 6월 30일부터 포트폴리오 구해서 23년 12월 28일까지 초과 수익률 구하기

In [30]:
EoMtemp = EoM[(EoM>=pd.Timestamp('2016-12-30'))] #& (EoM<=factor_ret_final.index[-1])]

In [31]:
# EoMtemp = EoM[(EoM>=pd.Timestamp('2016-12-30')) & (EoM<=pd.Timestamp('2024-12-30'))]

In [32]:
EoMtemp

array(['2017-01-31T00:00:00.000000000', '2017-02-28T00:00:00.000000000',
       '2017-03-31T00:00:00.000000000', '2017-04-28T00:00:00.000000000',
       '2017-05-31T00:00:00.000000000', '2017-06-30T00:00:00.000000000',
       '2017-07-31T00:00:00.000000000', '2017-08-31T00:00:00.000000000',
       '2017-09-29T00:00:00.000000000', '2017-10-31T00:00:00.000000000',
       '2017-11-30T00:00:00.000000000', '2017-12-28T00:00:00.000000000',
       '2018-01-31T00:00:00.000000000', '2018-02-28T00:00:00.000000000',
       '2018-03-30T00:00:00.000000000', '2018-04-30T00:00:00.000000000',
       '2018-05-31T00:00:00.000000000', '2018-06-29T00:00:00.000000000',
       '2018-07-31T00:00:00.000000000', '2018-08-31T00:00:00.000000000',
       '2018-09-28T00:00:00.000000000', '2018-10-31T00:00:00.000000000',
       '2018-11-30T00:00:00.000000000', '2018-12-28T00:00:00.000000000',
       '2019-01-31T00:00:00.000000000', '2019-02-28T00:00:00.000000000',
       '2019-03-29T00:00:00.000000000', '2019-04-30

In [33]:
EndOfMonthTemp = EoM[33]
NAVTemp = 100*10**8
# DailyNAV = dict()
ports = dict()
PortReturn = dict()
rebalancing = 1
table_ls=[]
for i in range(0, len(EoMtemp)-1, rebalancing): # 두달 주기로 리밸런싱하기
    date = EoMtemp[i]
    # it needs to check the second last business date 
    # if i==len(EoMtemp)-1:
    #     if EoMtemp[-2].month != EoMtemp[-1].month:
    #         break
    #     date = EoMtemp[-1] # date - np.timedelta64(1, 'D')
        
    print(date)
    # ESGDF = LoadESG(date, ESGDATADIR)
    kospi200atD = GetKOSPI200AtD(KOSPI200, date)
    A005930Weight = A005930Ratio.loc[date, 'ratio']
    ratio = ratioDataFrame.loc[date, 'Ratio']
    
    DataTable = GetDataTable(DataFrameDict, tradingDates, date, Dynamicportfolio, kospi200atD)
    RawTable = GetRawDataTable(DataTable, ratio)

    dd = date + np.timedelta64(-60, "D")
    dd = GetBusinessDayBefore(tradingDates, dd)
    flag2yr = 0
    if pd.Timestamp(date).year == pd.Timestamp(dd).year:
        FactorTable = GetFactorInitialTable(RawTable, ratio, flag2yr)
    else:
        flag2yr = 1
        FactorTable = GetFactorInitialTable(RawTable, ratio, flag2yr)

    FactorTable = AdjustLeverage(FactorTable)
    FactorTable = StandardizeColumns(FactorTable)
    table_ls.append(FactorTable)

2017-01-31T00:00:00.000000000
2017-02-28T00:00:00.000000000
2017-03-31T00:00:00.000000000
2017-04-28T00:00:00.000000000
2017-05-31T00:00:00.000000000
2017-06-30T00:00:00.000000000
2017-07-31T00:00:00.000000000
2017-08-31T00:00:00.000000000
2017-09-29T00:00:00.000000000
2017-10-31T00:00:00.000000000
2017-11-30T00:00:00.000000000
2017-12-28T00:00:00.000000000
2018-01-31T00:00:00.000000000
2018-02-28T00:00:00.000000000
2018-03-30T00:00:00.000000000
2018-04-30T00:00:00.000000000
2018-05-31T00:00:00.000000000
2018-06-29T00:00:00.000000000
2018-07-31T00:00:00.000000000
2018-08-31T00:00:00.000000000
2018-09-28T00:00:00.000000000
2018-10-31T00:00:00.000000000
2018-11-30T00:00:00.000000000
2018-12-28T00:00:00.000000000
2019-01-31T00:00:00.000000000
2019-02-28T00:00:00.000000000
2019-03-29T00:00:00.000000000
2019-04-30T00:00:00.000000000
2019-05-31T00:00:00.000000000
2019-06-28T00:00:00.000000000
2019-07-31T00:00:00.000000000
2019-08-30T00:00:00.000000000
2019-09-30T00:00:00.000000000
2019-10-31

In [34]:
tradingDates

array(['2010-03-04T00:00:00.000000000', '2010-03-05T00:00:00.000000000',
       '2010-03-08T00:00:00.000000000', ...,
       '2025-04-29T00:00:00.000000000', '2025-04-30T00:00:00.000000000',
       '2025-05-02T00:00:00.000000000'], dtype='datetime64[ns]')

In [35]:
# Datediff = [+1, -60, -180]
# # Get4Dates(Datediff, tradingDates, date)

# date_list = [date]
# for dd in Datediff:
#     d_temp = date + np.timedelta64(dd,'D')
#     if dd > 0:
#         Day = tradingDates[(tradingDates >= date) & (tradingDates <= date + np.timedelta64(20, 'D'))]
#         print(Day)
#         if len(Day) == 0:
#             raise ValueError("The given date is not found in the DataFrame.")

['2025-04-30T00:00:00.000000000' '2025-05-02T00:00:00.000000000']


In [37]:
# date = EoMtemp[-1]-np.timedelta64(1, 'D')
# # ESGDF = LoadESG(date, ESGDATADIR)
# kospi200atD = GetKOSPI200AtD(KOSPI200, date)
# A005930Weight = A005930Ratio.loc[date, 'ratio']
# ratio = ratioDataFrame.loc[date, 'Ratio']

# print(date, tradingDates)
# DataTable = GetDataTable(DataFrameDict, tradingDates, date, False, kospi200atD)
# RawTable = GetRawDataTable(DataTable, ratio)

# dd = date + np.timedelta64(-60, "D")
# dd = GetBusinessDayBefore(tradingDates, dd)
# flag2yr = 0
# if pd.Timestamp(date).year == pd.Timestamp(dd).year:
#     FactorTable = GetFactorInitialTable(RawTable, ratio, flag2yr)
# else:
#     flag2yr = 1
#     FactorTable = GetFactorInitialTable(RawTable, ratio, flag2yr)

# FactorTable = AdjustLeverage(FactorTable)
# FactorTable = StandardizeColumns(FactorTable)

In [ ]:
# table_ls.append(FactorTable)

In [38]:
df_list=[tab.reset_index().drop(columns='Name') for tab in table_ls]

In [39]:
df_list=[tab.set_index(pd.MultiIndex.from_arrays([[eom]*len(tab), tab['Symbol']], names=['Dates', 'Symbol'])) for i, (tab, eom) in enumerate(zip(df_list, EoMtemp))]

In [40]:
factor_df=pd.concat(df_list)

In [41]:
str(pd.to_datetime(EoMtemp[0]))[:10]

'2017-01-31'

In [42]:
factor_df

Symbol        BP    AssetP    SalesP       ROE   EBITDAY  \
Dates      Symbol                                                               
2017-01-31 A000030  A000030  2.045339       NaN       NaN       NaN       NaN   
           A000050  A000050  1.109435  1.120656 -0.579718 -0.293889  0.038626   
           A000070  A000070  0.861228 -0.146192  0.430815 -0.283817  0.281174   
           A000080  A000080 -0.244518  0.442812 -0.373087 -0.309334 -0.095967   
           A000100  A000100 -0.599844 -0.657295 -0.814438  0.000844 -0.629602   
...                     ...       ...       ...       ...       ...       ...   
2025-04-30 A402340  A402340  0.124467 -0.720280 -0.803449  1.047625  0.936104   
           A450080  A450080 -1.017086 -0.865050       NaN       NaN       NaN   
           A454910  A454910 -1.071064 -0.973271 -1.019973 -1.219030 -0.986860   
           A456040  A456040  0.791957  1.220708  0.893231  0.131194  0.802199   
           A457190  A457190 -1.094828 -0.920142       NaN       NaN       NaN   

                      EY_12M      EPSG       OPG        DY  InstituitionalBuy  \
Dates      Symbol                                                               
2017-01-31 A000030  1.190290       NaN       NaN       NaN          -0.067737   
           A000050       NaN       NaN       NaN -0.337365          -0.311508   
           A000070       NaN       NaN       NaN  0.058880          -0.729580   
           A000080 -1.206360 -0.155604 -0.351711  3.000000          -0.088817   
           A000100 -0.768735 -0.649415 -0.282151 -0.281906           0.745540   
...                      ...       ...       ...       ...                ...   
2025-04-30 A402340  3.000000 -0.463313 -0.241242 -0.937408          -0.212630   
           A450080       NaN       NaN       NaN -0.937408          -0.557719   
           A454910       NaN       NaN       NaN -0.937408          -0.465512   
           A456040       NaN       NaN       NaN  0.698196          -0.521984   
           A457190       NaN       NaN       NaN -0.937408          -0.353556   

                    Leverage       OPM    OP_M1M  ReverseMomentum  \
Dates      Symbol                                                   
2017-01-31 A000030       NaN       NaN  0.365002              NaN   
           A000050 -0.128141  0.349688       NaN         1.962856   
           A000070  3.000000 -0.116986 -3.000000         1.814071   
           A000080 -0.263575 -0.241787 -0.057302        -0.478739   
           A000100  1.327773 -0.172752  0.070327         0.144719   
...                      ...       ...       ...              ...   
2025-04-30 A402340  3.000000  3.000000  0.400206         0.179627   
           A450080  0.066743       NaN       NaN         3.000000   
           A454910  3.000000 -3.000000 -3.000000         2.542498   
           A456040  0.074730 -0.049681       NaN         0.639744   
           A457190 -0.074535       NaN       NaN         2.353519   

                    PriceMomentum  
Dates      Symbol                  
2017-01-31 A000030       2.011268  
           A000050      -0.319378  
           A000070      -0.742254  
           A000080      -0.234634  
           A000100      -1.691920  
...                           ...  
2025-04-30 A402340       0.172130  
           A450080      -1.447074  
           A454910      -0.762877  
           A456040      -0.665969  
           A457190       0.243232  

[20059 rows x 16 columns]

In [41]:
factor_df.to_pickle(output_dir+'/DQ_factor_'+str(pd.to_datetime(EoMtemp[-1]))[:10]+'.pkl')